This notebook is designed to prepare the data from the MEI.v2 folder (surface temperature, pressure, u and v wind components, and outgoing longwave radiation (OLR)) in a format that can be readily input into the multidimentsional positional encoding + informer for use.

In particular, the data are all put on a uniform 4 degree by 4 degree grid and are reported monthly starting January 1979 and ending December 2022. (Note: this requires taking the average of the OLR data, which is recorded daily in the MEI.v2 folder, for consistency with the other variables, which are recorded monthly.)

Variables are reported as 5 Numpy arrays (one for each variable) with shape latitude x longitude x time (months since January 1979). Only variables in the ENSO region of the Pacific (30°S–30°N and 100°E–70°W, the region used to calculate the MEI) are returned (ex. the Numpy arrays do not include the variables for the full globe).

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import netCDF4 as nc
import pygrib
from scipy.interpolate import RegularGridInterpolator

In [7]:
# READ IN THE DATA
# uses data in the MEI.v2 folder

years = np.arange(1979,2022,1,dtype=int)
years_str = years.astype(str)

num_yrs = len(years)

def open_anl_surf(variable, yrs):
    '''
    Opens the desired data (SST, 2 wind components, pressure) from the anl_surf folder for the desired years
    
    possible inputs for data (input as strings):
    - "pres": pressure
    - "ugrd": u component of wind
    - "vgrd": v component of wind
    - "tmp": surface temperature (assumed here to be SST)
    yrs is a list of strings denoting the desired years of data
    
    outputs data for that variable as an array of gribs
    '''
    num_yrs = len(years)
    
    path_pt1 = 'MEI.v2/ds628.1/anl_surf/'
    
    data = []
    
    for year in yrs:
        folder = path_pt1 + year + '/'
        search_for = "*" + variable + '*'
        file = glob(folder + search_for)[0]
        
        grbs = pygrib.open(file)
        
        data.append(grbs)
    
    return np.array(data)


def open_olr(yrs):
    '''
    opens + saves outgoing longwave radiation data from the olr folder
    
    yrs is a list of strings denoting the desired years of data
    
    outputs outgoing longwave radiation data as an array of NetCDF datasets
    '''
    file = 'MEI.v2/OLR_daily/olr-daily_v01r02_19790101_19791231.nc'
    
    num_yrs = len(years)
    
    path_pt1 = 'MEI.v2/OLR_daily/'
    
    data = []
    
    for year in yrs:
        folder = path_pt1 + year + '/'
        file = path_pt1 + 'olr-daily_v01r02_' +year+ '0101_' +year+ '1231.nc'
        
        ds = nc.Dataset(file)
        
        data.append(ds)
    
    return np.array(data)


# read in data here, save data as arrays of grbs or NetCDF DataSets
pres_data = open_anl_surf('pres', years_str)
ugrd_data = open_anl_surf('ugrd', years_str)
vgrd_data = open_anl_surf('vgrd', years_str)
tmp_data = open_anl_surf('tmp', years_str) # only NetCDF array

olr_data = open_olr(years_str)

In [8]:
def clean_nans(data):
    '''
    Replaces outliers (value 4.5 sigma above or below mean from whole period) values with 2D inerpolated values
    
    follows from Wolter and Timlin (1993)
    
    returns cleaned data
    '''
    # WAS NOT NEEDED--no NaN values found in MEI.v2 data!!
    
    return cleaned_data


In [10]:
# make uniform grid data from the grib files, save as a single array

def make4x4_grid_grib(grb):
    '''
    Puts data from a given grib object on a uniform 4 degree by 4 degree grid
    Follows from Wolter and Timlin (1993) and Wolter and Timlin (2011)
    
    returns 4x4 latitudes, longitudes, data
    
    '''
    
    # get latitude and longitude values
    data = grb.values
    lat = grb.latlons()[0][:,0]
    lon = grb.latlons()[1][0,:]
    
    # make an interpolating function for the original data
    interp = RegularGridInterpolator((lat, lon), data)
    
    # then data are interpolated on a 4 degree grid using the median values of the 2 degree grids
    latitude4 = np.flip(np.arange(-88.0, 90.0, 4.0)) # goes from 88 to -88
    longitude4 = np.arange(2.0, 360.0, 4.0) # only goes up to 358
    
    # longitude is like x coordinate, latitude is like y coordinate
    Lon4, Lat4 = np.meshgrid(longitude4, latitude4)
    
    data4 = interp((Lat4, Lon4))
    
    return Lat4, Lon4, data4

def construct_data_array_fromgrib(grbs_array):
    '''
    Constructs an array of data with each index alonh axis=2 corresponding to a month of data
    
    input grbs_array is the output from open_anl_surf for a given variable
    
    '''
    latitude4 = np.flip(np.arange(-88.0, 90.0, 4.0)) # goes from 88 to -88
    longitude4 = np.arange(2.0, 360.0, 4.0) # only goes up to 358.0
    lat_mask = (-30 <= latitude4) & (30 >= latitude4)
    lon_mask = (100 <= longitude4) & (290 >= longitude4)
    
    Lon4, Lat4 = np.meshgrid(longitude4, latitude4)
    
    # select ENSO region
    lat4_mask = (-30 <= Lat4) & (30 >= Lat4)
    lon4_mask = (100 <= Lon4) & (290 >= Lon4)
    ENSO_area_mask = lat4_mask & lon4_mask
    ENSO_3D_mask = np.repeat(ENSO_area_mask[:, :, np.newaxis], (num_yrs*12), axis=2)
    
    full_data_array = np.zeros((len(latitude4), len(longitude4), num_yrs*12))
    
    for i,grbs in enumerate(grbs_array):
        
        for j,grb in enumerate(grbs.select()):
            
            Lat, Lon, data = make4x4_grid_grib(grb)
            full_data_array[:,:,(12*i)+j] = data
    
    return np.reshape(full_data_array[ENSO_3D_mask], (len(latitude4[lat_mask]), len(longitude4[lon_mask]), (num_yrs*12)))

pressure = construct_data_array_fromgrib(pres_data)
temperature = construct_data_array_fromgrib(tmp_data)
uwind = construct_data_array_fromgrib(ugrd_data)
vwind = construct_data_array_fromgrib(vgrd_data)

In [12]:
def construct_seasons_from_days(data):
    '''
    Takes data in daily values NetCDF format, constructs it into sliding seasonal bins of data
    returns lat, lon, array of seasonal olr
    
    (seasons referring to the overlapping bimonthly seasons)
    '''
    # first just grab the lat and lon coordinates from the first NetCDF
    first_ds = data[0]
    lat = first_ds['lat'][:]
    lon = first_ds['lon'][:]
    
    # save all of the daily olr measurements to one big array!
    daily_data = np.zeros((len(lat), len(lon), 0))
    
    for ds in data:
        # get the data in the correct shape (lat, lon, day)
        olr = ds['olr'][:]
        olr1 = np.swapaxes(olr,0,1)
        olr2 = np.swapaxes(olr1,1,2)
        
        daily_data = np.concatenate((daily_data, olr2), axis=2)
    
    # add some extra zeros on the end as a buffer
    daily_data = np.concatenate((daily_data, np.zeros((len(lat), len(lon), 31))), axis=2)
    daily_data = daily_data[:,:,1:]
    
    # length of a month
    month_lens = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
    month_lens_lpyr = np.array([31,29,31,30,31,30,31,31,30,31,30,31])
    
    monthly_data = np.zeros((len(lat), len(lon), (num_yrs*12)))
    
    # track number of days past, starting at 31 because of the 0 padding
    day_tracker = 0
    
    # now we need to average the daily data into the months
    for i, year in enumerate(years):
        if year%4 == 0:
            months = month_lens_lpyr
        else:
            months = month_lens
        
        for j, month in enumerate(months):
            index = 12*i + j
            
            day_tracker += month
            monthly_data[:,:,index] = np.mean(daily_data[:,:,day_tracker-month:day_tracker], axis=2)
            
    return lat, lon, monthly_data

lat, lon, monthly_olr = construct_seasons_from_days(olr_data)

In [13]:
# make uniform grid data from the grib files, save as a single array

def make4x4_grid_olr(lat, lon, data):
    '''
    Puts data from a given season of OLR data on a uniform 4 degree by 4 degree grid
    
    parameters: latitude and longitude of OLR values (1D arrays) and a 2D array of one month of OLR data
    
    returns 4x4 latitudes, longitudes, data
    
    '''
    
    # make an interpolating function for the original data
    interp = RegularGridInterpolator((lat, lon), data)
    
    # then data are interpolated on a 4 degree grid using the median values of the 2 degree grids
    latitude4 = np.flip(np.arange(-88.0, 90.0, 4.0)) # goes from 88 to -88
    longitude4 = np.arange(2.0, 360.0, 4.0) # only goes up to 358
    
    # longitude is like x coordinate, latitude is like y coordinate
    Lon4, Lat4 = np.meshgrid(longitude4, latitude4)
    
    data4 = interp((Lat4, Lon4))
    
    return Lat4, Lon4, data4

def construct_data_array_olr(lat,lon,seasonal_data_array):
    '''
    Constructs an array of data with each index alonh axis=2 corresponding to a month of data
    
    input grbs_array is the output from open_anl_surf for a given variable
    
    '''
    latitude4 = np.flip(np.arange(-88.0, 90.0, 4.0)) # goes from 88 to -88
    longitude4 = np.arange(2.0, 360.0, 4.0) # only goes up to 358.0
    lat_mask = (-30 <= latitude4) & (30 >= latitude4)
    lon_mask = (100 <= longitude4) & (290 >= longitude4)
    
    Lon4, Lat4 = np.meshgrid(longitude4, latitude4)
    
    # select ENSO region
    lat4_mask = (-30 <= Lat4) & (30 >= Lat4)
    lon4_mask = (100 <= Lon4) & (290 >= Lon4)
    ENSO_area_mask = lat4_mask & lon4_mask
    ENSO_3D_mask = np.repeat(ENSO_area_mask[:, :, np.newaxis], (num_yrs*12), axis=2)
    
    processed_data = np.zeros(ENSO_3D_mask.shape)
    
    for i in range(seasonal_data_array.shape[2]):
        olr = seasonal_data_array[:,:,i]
        Lat, Lon, data = make4x4_grid_olr(lat,lon,olr)
        processed_data[:,:,i] = data
            
    
    return np.reshape(processed_data[ENSO_3D_mask], (len(latitude4[lat_mask]), len(longitude4[lon_mask]), (num_yrs*12)))

olr = construct_data_array_olr(lat,lon,monthly_olr)


In [14]:
def calculate_standardized_anomoly(data):
    '''
    Calculates standardized anomoly for given data (only parameter)
    Data should already be in offset month bins (Dec-Jan, Jan-Feb, etc.) and normal 4x4 grid spacing
    
    standardized anomoly = (value - avg value for given time range) / standard deviation in given time range
    
    '''
    mean = np.mean(data, axis=2)
    stdev = np.std(data, axis=2)
    
    mean_array = np.repeat(mean[:, :, np.newaxis], data.shape[2], axis=2)
    stdev_array = np.repeat(stdev[:, :, np.newaxis], data.shape[2], axis=2)
    
    standardized_anomoly = data
    
    standardized_anomoly -= mean_array
    standardized_anomoly /= stdev_array
    
    return standardized_anomoly

standardized_pressure = calculate_standardized_anomoly(pressure)
standardized_temperature = calculate_standardized_anomoly(temperature)
standardized_uwind = calculate_standardized_anomoly(uwind)
standardized_vwind = calculate_standardized_anomoly(vwind)
standardized_olr = calculate_standardized_anomoly(olr)

In [18]:
# Save the processed variable data, if desired
# This will save the standardized anomolies of the variables, but perhaps we want the regular values (not standardized) for prediction

np.save('standardized_pressure.npy',standardized_pressure)
np.save('standardized_temperature.npy',standardized_temperature)
np.save('standardized_uwind.npy',standardized_uwind)
np.save('standardized_vwind.npy',standardized_vwind)
np.save('standardized_olr.npy',standardized_olr)